In [1]:
import requests
import json
from bs4 import BeautifulSoup
import re

In [2]:
quora_url = 'https://www.quora.com/'

In [6]:
r = requests.get(quora_url+'topic/china')
r.status_code

200

In [19]:
soup = BeautifulSoup(r.content,'lxml')
topic_json = {}

In [9]:
feed_questions = []
for a in soup.find_all('a',attrs={'class':'question_link'}):
    feed_questions.append({'question_link':a['href'],'question_text':a.text})
    #questions.extend(re.findall('[0-9]{1}([^\?0-9]+\?)',p.get_text(),re.M))

In [20]:
topic_json['feed_questions'] = feed_questions

In [22]:
topic_json['TopicWikiText'] = soup.find_all('div',attrs={'class':'TopicWikiText'})[0].text

In [24]:
Questions = soup.find_all('a',attrs={'class':'StatsRow TopicQuestionsStatsRow'})[0]
topic_json['Questions_link'] = Questions['href']
topic_json['Questions_text'] = Questions.text

In [26]:
Followers = soup.find_all('a',attrs={'class':'TopicFollowersStatsRow StatsRow'})[0]
topic_json['Followers_link'] = Followers['href']
topic_json['Followers_text'] = Followers.text

In [27]:
Edits = soup.find_all('a',attrs={'class':'TopicEditsStatsRow StatsRow'})[0]
topic_json['Edits_link'] = Edits['href']
topic_json['Edits_text'] = Edits.text

In [31]:
related_topics = []#RelatedTopicsListItem HoverMenu
for a in soup.find_all('a',attrs={'class':'RelatedTopicsListItem HoverMenu'}):
    related_topics.append({'topic_link':a['href'],'TopicName':a.find_all('span',attrs={'class':'TopicName'})[0].text})
topic_json['related_topics'] = related_topics

In [126]:
class Topic(object):
    def __init__(self,topic_name):
        self.topic_name = topic_name
        self.url = 'https://www.quora.com/topic/'+self.topic_name
        self.res = requests.get(self.url)
        if self.res.status_code == 200:
            self.soup = BeautifulSoup(self.res.content,'lxml')   
        self.feed_questions = []
        self.Questions = {}
        self.Followers = {}
        self.Edits = {}
        self.related_topics = []
        for a in self.soup.find_all('a',attrs={'class':'question_link'}):
            self.feed_questions.append({'question_link':a['href'],'question_text':a.text})
        Questions_soup = self.soup.find_all('a',attrs={'class':'StatsRow TopicQuestionsStatsRow'})[0]
        self.Questions['Questions_link'] = Questions_soup['href']
        self.Questions['Questions_text'] = Questions_soup.text
        Followers_soup = self.soup.find_all('a',attrs={'class':'StatsRow TopicQuestionsStatsRow'})[0]
        self.Followers['Followers_link'] = Followers_soup['href']
        self.Followers['Followers_text'] = Followers_soup.text
        Edits_soup = self.soup.find_all('a',attrs={'class':'StatsRow TopicQuestionsStatsRow'})[0]
        self.Edits['Edits_link'] = Edits_soup['href']
        self.Edits['Edits_text'] = Edits_soup.text
        for a in self.soup.find_all('a',attrs={'class':'RelatedTopicsListItem HoverMenu'}):
            self.related_topics.append({'topic_link':a['href'],'TopicName':a.find_all('span',attrs={'class':'TopicName'})[0].text})
        self.topic_json = {}
        
    def get_topic_json(self):
        self.topic_json['topic_name'] = self.topic_name
        self.topic_json['topic_url'] = self.url
        self.topic_json['feed_questions'] = self.feed_questions
        self.topic_json['TopicWikiText'] = self.soup.find_all('div',attrs={'class':'TopicWikiText'})[0].text
        self.topic_json['Questions'] = self.Questions
        self.topic_json['Followers'] = self.Followers
        self.topic_json['Edits'] = self.Edits
        self.topic_json['related_topics'] = self.related_topics
        return self.topic_json

In [106]:
china = Topic('china')

In [107]:
j = china.get_topic_json()

In [108]:
china.get_topic_json()

{'Edits': {'Edits_link': '/topic/China/all_questions',
  'Edits_text': u'78.4kQuestions'},
 'Followers': {'Followers_link': '/topic/China/all_questions',
  'Followers_text': u'78.4kQuestions'},
 'Questions': {'Questions_link': '/topic/China/all_questions',
  'Questions_text': u'78.4kQuestions'},
 'TopicWikiText': u"Topic mainly concerns the East Asian state the People's Republic of China.For the Republic of China see Taiwan",
 'feed_questions': [{'question_link': '/How-safe-is-China',
   'question_text': u'How safe is China?'},
  {'question_link': '/How-do-foreigners-view-China-after-visiting-China',
   'question_text': u'How do foreigners view China after visiting China?'},
  {'question_link': '/Why-do-the-Chinese-still-care-about-the-Nanjing-massacre-so-much',
   'question_text': u'Why do the Chinese still care about the Nanjing massacre so much?'},
  {'question_link': '/Why-is-China-still-so-hateful-and-rude-to-current-Japan-when-they-had-no-part-in-the-militarys-past',
   'question

In [144]:
class Question(object):
    def __init__(self,questoin_name):
        self.questoin_name = questoin_name
        self.url = 'https://www.quora.com/'+self.questoin_name
        self.res = requests.get(self.url)
        if self.res.status_code == 200:
            self.soup = BeautifulSoup(self.res.content,'lxml')
        self.QuestionTopic = []
        for a in self.soup.find_all('a',attrs={'class':'TopicNameLink HoverMenu topic_name'}):
            self.QuestionTopic.append({'topic_link':a['href'],'TopicName':a.text})
        self.answer_count = int(re.findall('[0-9]+',self.soup.find_all('div',attrs={'class':'answer_count'})[0].text)[0])
        self.answer = []
        for d1 in self.soup.find_all('div',attrs={'class':'AnswerBase'}):
            for d2 in d1.find_all('div',attrs={'class':'AnswerHeader ContentHeader'}):
                user = {}
                for a in d2.find_all('a',attrs={'class':'user'}):
                    user['user_link'] = a['href']
                    user['UserName'] = a.text
            self.answer.append({'user':user,'answer_text':d1.find_all('span',attrs={'class':'rendered_qtext'})[0].text})
        self.question_json = {}
        
    def get_question_json(self):
        self.question_json['questoin_name'] = self.questoin_name
        self.question_json['url'] = self.url
        self.question_json['QuestionTopic'] = self.QuestionTopic
        self.question_json['answer_count'] = self.answer_count
        self.question_json['answer'] = self.answer
        return self.question_json

In [69]:
res = requests.get('https://www.quora.com/How-do-mongolians-treat-foreigners')

In [70]:
q_soup = BeautifulSoup(res.content,'lxml')

In [71]:
for a in q_soup.find_all('a',attrs={'class':'TopicNameLink HoverMenu topic_name'}):
    print a

<a action_mousedown="TopicLinkClickthrough" class="TopicNameLink HoverMenu topic_name" href="/topic/Mongolian-Ethnicity-and-People" id="__w2_wrzlw0I_link" target=""><span class="name_text"><span id="CwXyHY"><span class="TopicNameSpan TopicName" id="__w2_DDEHbmc_card">Mongolian Ethnicity and People</span></span></span></a>
<a action_mousedown="TopicLinkClickthrough" class="TopicNameLink HoverMenu topic_name" href="/topic/Mongolia" id="__w2_XOvZkfu_link" target=""><span class="name_text"><span id="Swupoo"><span class="TopicNameSpan TopicName" id="__w2_f2lQpgy_card">Mongolia</span></span></span></a>
<a action_mousedown="TopicLinkClickthrough" class="TopicNameLink HoverMenu topic_name" href="/topic/Foreigners" id="__w2_QBEYCFM_link" target=""><span class="name_text"><span id="JalypP"><span class="TopicNameSpan TopicName" id="__w2_Co3shmF_card">Foreigners</span></span></span></a>
<a action_mousedown="TopicLinkClickthrough" class="TopicNameLink HoverMenu topic_name" href="/topic/Ethnic-and-C

In [102]:
# pagedlist_item AnswerBase AnswerHeader ContentHeader feed_item_answer_user Answer
answer = []
for d1 in q_soup.find_all('div',attrs={'class':'AnswerBase'}):
    for d2 in d1.find_all('div',attrs={'class':'AnswerHeader ContentHeader'}):
        user = {}
        for a in d2.find_all('a',attrs={'class':'user'}):
            user['user_link'] = a['href']
            user['UserName'] = a.text
    answer.append({'user':user,'answer_text':d1.find_all('span',attrs={'class':'rendered_qtext'})[0].text})

In [103]:
answer

[{'answer_text': u"mongols are highly hospitable people. for us, guest is above everything else. this is especially true in countryside. when somebody comes to your house, the usual reaction is to feed the person and give tea. and afterwards ask what that person is doing there etc. sleeping in the house is common.mongolia is generally safe country for foreigners as well.there are couple of taboos that i wrote previously. have alook:Anandsaikhan Nyamdavaa's answer to What are the biggest mistakes people make when visiting Mongolia?",
  'user': {'UserName': u'Anandsaikhan Nyamdavaa',
   'user_link': '/profile/Anandsaikhan-Nyamdavaa'}},
 {'answer_text': u'If you\u2019re generalizing,Some of us can be a little too rude to foreigners. On the other hand, some of us treat foreigners as a sack of cash and charge extra for any services. Some of us, who are just as innocent and friendly as Mongolians are known for, look favorably upon foreigners and are happy to see many foreigners choosing our 

In [114]:
int(q_soup.find_all('div',attrs={'class':'answer_count'})[0].text.split(' ')[0])

2

In [124]:
q = Question('Do-Chinese-and-Japanese-have-similarities')

In [125]:
q.get_question_json()

{'QuestionTopic': [{'TopicName': u'China and Japan',
   'topic_link': '/topic/China-and-Japan-1'},
  {'TopicName': u'Japanese Ethnicity and People',
   'topic_link': '/topic/Japanese-Ethnicity-and-People'},
  {'TopicName': u'Japanese (language)',
   'topic_link': '/topic/Japanese-language'},
  {'TopicName': u'Chinese (language)',
   'topic_link': '/topic/Chinese-language'},
  {'TopicName': u'Chinese Ethnicity and People',
   'topic_link': '/topic/Chinese-Ethnicity-and-People-1'},
  {'TopicName': u'Japan', 'topic_link': '/topic/Japan'},
  {'TopicName': u'China', 'topic_link': '/topic/China'}],
 'answer': [{'answer_text': u'They have, both culturally and in their languages. Some are almost the same, some are similar and others are completely different. Still, what you want to know specifically? There\u2019s a lot of variable to nations and their people.',
   'user': {'UserName': u'Rodrigo Xavier',
    'user_link': '/profile/Rodrigo-Xavier-1'}},
  {'answer_text': u'Yes, there are a lot of

In [128]:
china_topic = Topic('china')

In [132]:
china_topic_json = china_topic.get_topic_json()

In [134]:
china_feed_questions = china_topic_json['feed_questions']

In [138]:
china_feed_questions_name = [question['question_link'][1:].encode('utf-8') for question in china_feed_questions]

In [139]:
china_feed_questions_name

['How-safe-is-China',
 'How-do-foreigners-view-China-after-visiting-China',
 'Why-do-the-Chinese-still-care-about-the-Nanjing-massacre-so-much',
 'Should-China-get-out-of-Tibet-like-UK-got-out-of-Hong-Kong',
 'Why-is-China-still-so-hateful-and-rude-to-current-Japan-when-they-had-no-part-in-the-militarys-past',
 'Why-do-educated-Chinese-support-CCP-despite-not-having-the-freedom-to-criticize-Chinese-politicians',
 'How-do-foreigners-view-China-after-visiting-China',
 'What-does-\xe6\x88\x91\xe5\x8e\xbb-mean-in-Chinese',
 'What-does-it-feel-like-to-be-a-woman-living-in-China',
 'How-do-foreigners-view-China-after-visiting-China',
 'What-do-non-Chinese-Japanese-people-think-of-the-Nanjing-Massacre-2',
 'What-do-non-Chinese-Japanese-people-think-of-the-Nanjing-Massacre?no_redirect=1',
 'Why-is-it-that-people-are-saying-China-is-a-good-place-to-live-in-Millions-of-people-have-been-migrating-from-China-to-other-countries',
 'Is-Chinese-a-beautiful-language',
 'Why-dont-Chinese-people-attack-

In [140]:
for feed_question in china_feed_questions_name[:1]:
    print feed_question

How-safe-is-China


In [145]:
q = Question('How-safe-is-China')

In [147]:
q_json = q.get_question_json()

In [150]:
answers = q_json['answer']

In [151]:
corpus = ''
for a in answers:
    corpus = corpus+'\n'+a['answer_text']

In [153]:
corpus

u'\nI have been in Beijing for more than 15 years and not once have I felt threatened in any way. Beijing have always been a safe place from my perspective. One thing that I have come to understand is that if you are a foreigner/expat in China, most locals are nice and helpful both in big cities and small towns as well. Generally, rural China is considered safe by many Chinese. Big cities, especially Beijing and Shanghai- have their headaches - pick-pocketing mostly, but generally safe. Guandong province is notoriously known for its not-so-safe problems (Police pledge to fight gangs in Guangzhou\'s townships and villages). The hotbed of identity theft, bank card hacking and copying is probably in Dongguan city also in Guangzhou.I once got my passport stolen in Shenzhen (in 2001) when dropping off the airport bus. In those days, the stops for airport buses were not regulated at all. The moment you drop-off, you get this crowd of teenagers swarming towards you with business cards and at 